# Recursive descent parser + AST builder + interpreter
Template notebook.


In [39]:
# Νίκος Λιθαρής
# ΑΜ: Π2019083

from io import StringIO
import plex
from compilerlabs import ASTNode

In [40]:
# parsing error, a user-defined exception
class ParseError(Exception):
    pass

In [41]:
class MyParser():
    def __init__(self, scanner):
        self.scanner = scanner
        self.next_token = None
        self.next_lexeme = None

    def match(self, expected):
        if self.next_token == expected:
            self.next_token, self.next_lexeme = self.scanner.read()
        else:
            raise ParseError('Expected {}, found {} instead'.format(
                expected, self.next_token))

    def parse(self):
        self.next_token, self.next_lexeme = self.scanner.read()
        sl = self.stmt_list()
        self.match(None)
        return sl

    def stmt_list(self):
        if self.next_token in ('ID', 'print'):
            s = self.stmt()
            sl = self.stmt_list()
            if sl is None:
                return s
            return ASTNode(subnodes=[s, sl], attributes={'type': '.'})
        elif self.next_token is None:
            return
        else:
            raise ParseError('stmt_list: Parse Error')

    def stmt(self):
        if self.next_token == 'ID':
            val = self.next_lexeme
            self.match('ID')
            self.match('=')
            exp = self.expr()
            return ASTNode(subnodes=[exp], attributes={'type': 'ASSIGN', 'name': val})
        elif self.next_token == 'print':
            self.match('print')
            exp = self.expr()
            return ASTNode(subnodes=[exp], attributes={'type': 'PRINT'})
        else:
            raise ParseError('stmt: Parse Error')

    def expr(self):
        t = self.term()
        tt = self.expr_tail()
        if tt is None:
            return t
        return ASTNode(subnodes=[t, tt[1]], attributes={'type': 'OPERATOR', 'func': tt[0]})

    def expr_tail(self):
        if self.next_token in ('+', '-', '*', '/', '%', '^'):
            op = self.addop()
            t = self.term()
            tt = self.expr_tail()
            if tt is None:
                return op, t
            n = ASTNode(subnodes=[t, tt[1]], attributes={
                        'type': 'OPERATOR', 'func': tt[0]})
            return op, n
        elif self.next_token in ('ID', 'print', ')', None):
            return

    def term(self):
        f = self.factor()
        ft = self.term_tail()
        if ft is None:
            return f
        return ASTNode(subnodes=[f, ft[1]], attributes={'type': 'OPERATOR', 'func': ft[0]})

    def term_tail(self):
        if self.next_token in ('*', '/', '%', '^'):
            op = self.multop()
            f = self.factor()
            ft = self.term_tail()
            if ft is None:
                return op, f
            n = ASTNode(subnodes=[f, ft[1]], attributes={
                        'type': 'OPERATOR', 'func': ft[0]})
            return op, n
        elif self.next_token in ('+', '-', 'ID', 'print', ')', None):
            return

    def factor(self):
        if self.next_token == '(':
            self.match('(')
            exp = self.expr()
            self.match(')')
            return exp
        elif self.next_token == 'ID':
            val = self.next_lexeme
            self.match('ID')
            return ASTNode(attributes={'type': 'VAR', 'name': val})
        elif self.next_token == 'NUMBER':
            val = self.next_lexeme
            self.match('NUMBER')
            return ASTNode(attributes={'type': 'NUMBER', 'value': float(val)})
        else:
            raise ParseError('factor: Parse Error')

    def addop(self):
        if self.next_token in ('+', '-', '*', '/', '%', '^'):
            op = self.next_token
            self.match(op)
            return op
        else:
            raise ParseError('addop: Parse Error')

    def multop(self):
        if self.next_token in ('*', '/', '%', '^'):
            op = self.next_token
            self.match(op)
            return op
        else:
            raise ParseError('multop: Parse Error')


In [42]:
# runtime error, a user-defined exception
class RunError(Exception):
    pass

In [43]:
# class of AST walking interpreter
class MyInterpreter():

    def __init__(self):
        
        # TODO: replace with any initial interpreter state, e.g. set storage space for variables etc
        self.symbol_table = {}


    def run(self, root_node):
        # begin AST walking at root node
        if root_node is not None:    # if not an empty AST
            self.execute(root_node) 
            pass
    
    def execute(self, node):
        if node.attributes['type'] == 'ASSIGN':
            self.symbol_table[node.attributes['name']] = self.compute(node.subnodes[0])
        elif node.attributes['type'] == 'PRINT':
            print(self.compute(node.subnodes[0]))
        else:
            self.execute(node.subnodes[0])
            self.execute(node.subnodes[1])
    
    
    
    def compute(self, node):
        if node.attributes['type'] == 'NUMBER':
            return node.attributes['value']
        elif node.attributes['type'] == 'VAR':
            var_name = node.attributes['name']
            if var_name in self.symbol_table:
                return self.symbol_table[var_name]
            else:
                raise RunError('Variable name was not found')
        else:
            a = self.compute(node.subnodes[0])
            b = self.compute(node.subnodes[1])
            if node.attributes['func'] == '+':
                return a + b
            elif node.attributes['func'] == '-':
                return a - b
            elif node.attributes['func'] == '*':
                return a * b
            elif node.attributes['func'] == '/':
                return a / b
            elif node.attributes['func'] == '%':
                return a % b
            elif node.attributes['func'] == '^':
                return a ** b
        
        

In [44]:
# main part of program

# TODO: define patterns for plex here
digit = plex.Range('09')
letter = plex.Range('AZaz')
underscore = plex.Str('_')
dot = plex.Str('.')
af = plex.Range('AFaf')
operator = plex.Any('+-*/=()%^')
spaces = plex.Rep1(plex.Any(' \t\n'))

float1 = plex.Rep1(digit) + dot + plex.Rep1(digit)
int1 = plex.Rep1(digit)
number = (float1|int1)
ident = (letter|underscore) + plex.Rep(letter|digit|underscore)

printf = plex.Str('print|%')



# TODO: create plex lexicon
kw = plex.Str('print')
lexicon = plex.Lexicon([(kw, 'print'), (number, 'NUMBER'), (operator, plex.TEXT), (spaces, plex.IGNORE), (ident, 'ID')])
    
    
# TODO: define input text
text = """
print 5-3-2
print 12 / 3 / 2
print 1+2*3
print 1/2^4
print 7 % 3
print 2 ^ 3 ^ 2
print 5 * 4 % 3 + 2 - 6 / 3
print 10 ^ 2 % 7 * 5 - 3 / 2
"""
    
# create plex scanner for input text
scanner = plex.Scanner(lexicon,StringIO(text))

# create recursive descent parser/AST generator
parser = MyParser(scanner)

try:
    ast = parser.parse()
    print(ast)
    
except ParseError as e:
    _,lineno,charno = scanner.position()
    print('Syntax error at line:{} char:{}, {}'.format(lineno,charno+1,e))
            
except plex.errors.PlexError:            
    _,lineno,charno = scanner.position()
    print("Scanner Error at line {} char {}".format(lineno,charno+1))

else:   # if no scanner or syntax error
    
    # create AST interpreter
    runner = MyInterpreter()

    try:
        runner.run(ast)

    except RunError as e:
        print('Runtime error: {}'.format(e))


ASTNode{'type': '.'}
├── ASTNode{'type': 'PRINT'}
│   └── ASTNode{'type': 'OPERATOR', 'func': '-'}
│       ├── ASTNode{'type': 'NUMBER', 'value': 5.0}
│       └── ASTNode{'type': 'OPERATOR', 'func': '-'}
│           ├── ASTNode{'type': 'NUMBER', 'value': 3.0}
│           └── ASTNode{'type': 'NUMBER', 'value': 2.0}
└── ASTNode{'type': '.'}
    ├── ASTNode{'type': 'PRINT'}
    │   └── ASTNode{'type': 'OPERATOR', 'func': '/'}
    │       ├── ASTNode{'type': 'NUMBER', 'value': 12.0}
    │       └── ASTNode{'type': 'OPERATOR', 'func': '/'}
    │           ├── ASTNode{'type': 'NUMBER', 'value': 3.0}
    │           └── ASTNode{'type': 'NUMBER', 'value': 2.0}
    └── ASTNode{'type': '.'}
        ├── ASTNode{'type': 'PRINT'}
        │   └── ASTNode{'type': 'OPERATOR', 'func': '+'}
        │       ├── ASTNode{'type': 'NUMBER', 'value': 1.0}
        │       └── ASTNode{'type': 'OPERATOR', 'func': '*'}
        │           ├── ASTNode{'type': 'NUMBER', 'value': 2.0}
        │           └── ASTNode